In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

In [3]:
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [11]:

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

cache_store = LocalFileStore("./cache/")

In [12]:
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, api_key=OPENAI_API_KEY)
# llm = Cohere(temperature=0)
llm = Ollama(model="zephyr")

In [13]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 239.03it/s]


In [14]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # model_name="gpt-3.5",
    chunk_size=1000,
    chunk_overlap=200,
)
chunked = text_splitter.split_documents(docs)
print(len(chunked))

33


## OpenAI Embeddings


In [15]:
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace=embeddings_model.model)

In [16]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 186 ms, sys: 18.3 ms, total: 204 ms
Wall time: 1.6 s


In [40]:

# retriever = db.as_retriever()
retriever = db.as_retriever(search_type="mmr")

retrieved_docs = retriever.get_relevant_documents("How do I deal with Enhanced Grims?")
pretty_print_docs(retrieved_docs)

Document 1:

These monsters are considered the teenagers of the species and are among the most mobile denizens of the nine realms. Be careful, though, as they will strike with powerful melee attacks. Remember, when attacking enemies with blockable attacks, tap L1 at the last second to parry an incoming attack and apply a temporary stun effect – this will leave the enemy vulnerable to a flurry of quick attacks.

Having taken down the Grim, pick up any loot that they may have dropped (such as crafting materials, Healthstones, etc.) and head to the nearby dock and climb the wooden structure to loot the Red Coffin, which is located on the ledge above, to receive +2 Forged Iron and some Hacksilver.

Return to the zipline and jump down the nearby ledge. From here, proceed to climb up the chain that’s dangling from the wooden structure to reach the broken wheel – before interacting with the wheel, be sure to loot the Hacksilver chest that’s found not too far from Atreus.
Having turned the bro

In [41]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "How do I deal with Enhanced Grims?"
)
pretty_print_docs(compressed_docs)

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

To deal with Enhanced Grims, it is recommended to tap L1 at the last second while attacking enemies with blockable attacks to parry their incoming attacks and apply a temporary stun effect, leaving them vulnerable to quick attacks. This technique can be useful when facing these mobile denizens of the nine realms, who strike with powerful melee attacks and should be handled with caution. Additionally, it is suggested to loot any crafting materials, Healthstones, or Hacksilver that Enhanced Grims may drop upon defeating them.
----------------------------------------------------------------------------------------------------
Document 2:

Question: How do I deal with Enhanced Grims?

Relevant context: None, the given question does not appear in the provided context. Return NO_OUTPUT.
----------------------------------------------------------------------------------------------------
Document 3:

Question: How do I deal with Enhanced Grims?

Relevant parts:
- "Cursed Grims fro

In [42]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)
chain({"query": "How do I deal with Enhanced Grims?"})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': 'How do I deal with Enhanced Grims?',
 'result': 'The provided context does not offer any information on how to deal with Enhanced Grims. Based on the description of these enemies as "the teenagers of the species" and "among the most mobile denizens of the nine realms with powerful melee attacks," it\'s recommended to parry their blockable attacks by tapping L1 at the last second, which will apply a temporary stun effect and leave them vulnerable to quick attacks. However, there is no further information provided about their behavior or weaknesses beyond this basic strategy. It\'s possible that additional context could offer more details on how to effectively defeat Enhanced Grims, but without further information it\'s impossible to provide specific strategies or tactics for dealing with these enemies.'}

In [45]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "How do I deal with Enhanced Grims?"
)
pretty_print_docs(compressed_docs)

Document 1:

These monsters are considered the teenagers of the species and are among the most mobile denizens of the nine realms. Be careful, though, as they will strike with powerful melee attacks. Remember, when attacking enemies with blockable attacks, tap L1 at the last second to parry an incoming attack and apply a temporary stun effect – this will leave the enemy vulnerable to a flurry of quick attacks.

Having taken down the Grim, pick up any loot that they may have dropped (such as crafting materials, Healthstones, etc.) and head to the nearby dock and climb the wooden structure to loot the Red Coffin, which is located on the ledge above, to receive +2 Forged Iron and some Hacksilver.

Return to the zipline and jump down the nearby ledge. From here, proceed to climb up the chain that’s dangling from the wooden structure to reach the broken wheel – before interacting with the wheel, be sure to loot the Hacksilver chest that’s found not too far from Atreus.
Having turned the bro

In [46]:

chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)
chain({"query": "How do I deal with Enhanced Grims?"})

{'query': 'How do I deal with Enhanced Grims?',
 'result': "To deal with Enhanced Grims, you can use a variety of strategies. Here are some tips:\n\n1. Use the Blades of Chaos to stun them: One effective way to take down Enhanced Grims is by stunning them with your Blades of Chaos. When they're stunned, you can deal heavy damage without having to worry about their poisonous saliva.\n\n2. Keep moving: Enhanced Grims have a wide range attack, so it's essential to keep moving and avoid getting cornered. This will also prevent them from being able to spit at you continuously.\n\n3. Use ranged attacks: The Leviathan Axe and the Blades of Chaos both have ranged attacks that can be used to take down Enhanced Grims from a distance. This is especially useful if there are multiple grims in the area, as it allows you to thin out their numbers before engaging them in melee combat.\n\n4. Use elemental weapons: Elemental weapons like the Frozen Axe and the Blaze of Set can be effective against Enhan

In [126]:
query = "How do I deal with Enhanced Grims?"
search_result = db.similarity_search(query)
# pretty_print_docs(search_result)
# search_result[0].page_content

In [32]:


llm.invoke("Tell me a joke")

'Why did the tomato turn red?\nBecause it saw the salad dressing and didn\'t want to be dressed!\n(This is a play on the phrase "seeing red" which means feeling very angry.)'